# 주류 도매업 크롤링(2 ~ 3페이지)

In [1]:
import numpy as np
import pandas as pd
from selenium import webdriver #Selenium의 웹 드라이버를 사용하기 위한 모듈을 임포트
from selenium.webdriver.common.by import By #Selenium에서 사용하는 By 클래스를 임포트합. 웹 요소를 검색하는데 사용.
from selenium.webdriver.common.keys import Keys #키보드 입력 제어를 위해 Keys 클래스 임포트
from selenium.webdriver.chrome.service import Service #Chrome 드라이버 서비스를 사용하기 위한 모듈 임포트
from selenium.webdriver.chrome.options import Options #Chrome 드라이버 옵션을 설정하기 위한 클래스 임포트
from webdriver_manager.chrome import ChromeDriverManager #Chrome 드라이버를 자동으로 설치 및 관리하는데 사용되는 드라이버 매니저 임포트
from bs4 import BeautifulSoup #Beautiful soup 크롤링을위해
import time

In [7]:

# 브라우저 꺼짐 방지
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
# 불필요한 에러 메시지 없애기
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://dart.fss.or.kr/dsae001/main.do')
driver.maximize_window()
driver.implicitly_wait(5)
# 업종별 탭
upjong_btn=driver.find_element(By.XPATH,'//*[@id="businessTab"]/a')
time.sleep(2)
upjong_btn.click()
# 최상위 분류 : 도매 및 상품 중개업
type_btn_1=driver.find_element(By.XPATH,'//*[@id="root4646"]/i')
type_btn_1.click()
time.sleep(1)
# 두 번째 상위 분류 : 도매 및 상품 중개업
type_btn_2=driver.find_element(By.XPATH,'//*[@id="46"]/i')
type_btn_2.click()
time.sleep(1)
# 세 번째 상위 분류 : 음,식료품 및 담배 도매업
type_btn_3=driver.find_element(By.XPATH,'//*[@id="463"]/i')
type_btn_3.click()
time.sleep(1)
# 네 번재 상위 분류 : 음료 및 담배 도매업
type_btn_4=driver.find_element(By.XPATH,'//*[@id="4633"]/i')
type_btn_4.click()
time.sleep(1)
# 마지막 분류 : 주류 도매업
type_btn_end=driver.find_element(By.XPATH,'//*[@id="46331_anchor"]')
type_btn_end.click()
time.sleep(1)
time.sleep(3)
table_list=[]
comp_name=[]
df_list=[]
page_list=driver.find_elements(By.CSS_SELECTOR,'#listContents > div.psWrap > div.pageSkip > ul > li')
for i in range(2,4):
    driver.find_element(By.XPATH,f'//*[@id="listContents"]/div[2]/div[2]/ul/li[{i}]/a').click()
    # 회사 개수 파악
    comp_list=driver.find_elements(By.XPATH,'//*[@id="corpTabel"]/tbody/tr')
    time.sleep(2)
    for j in range(len(comp_list)):
        # 회사 선택
        driver.find_element(By.XPATH,f'//*[@id="corpTabel"]/tbody/tr[{j+1}]/td[1]/span/a').click()
        time.sleep(1)
        
        # BeautifulSoup으로 회사 정보 테이블만 추출
        html=driver.page_source
        soup=BeautifulSoup(html,'html.parser')
        time.sleep(1)
        
        # 회사 정보 리스트에 저장
        inform_table=soup.select('#corpDetailTabel > tbody')[0]
        table_list.append(inform_table)
        time.sleep(1)
        
        # 회사 정보 테이블 기반 데이터프레임 구축
        comp_name=[]
        owner_name=[]
        address=[]
        home_page=[]
        phone_num=[]
        fax_num=[]
        comp_type=[]
        start_date=[]
        data_list=[]
    for j in table_list:
        # 공백 제거
        comp_name.append(j.select('tr>td')[2].text.strip())
        owner_name.append(j.select('tr>td')[4].text.split('\n')[0].lstrip())
        address.append(j.select('tr>td')[8].text.strip())
        home_page.append(j.select('tr>td')[9].text.strip())
        phone_num.append(j.select('tr>td')[11].text)
        fax_num.append(j.select('tr>td')[12].text)
        comp_type.append(j.select('tr>td')[13].text)
        
        # 날짜형으로 변환
        start_date.append(pd.to_datetime(j.select('tr>td')[14].text,format='%Y-%m-%d'))
    df_raw=pd.DataFrame({
            '회사명':comp_name,
            '대표자명':owner_name,
            '주소':address,
            '홈페이지':home_page,
            '전화번호':phone_num,
            '팩스번호':fax_num,
            '업종':comp_type,
            '설립일':start_date
        })
    df_list.append(df_raw)
driver.close()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="corpTabel"]/tbody/tr[23]/td[1]/span/a"}
  (Session info: chrome=122.0.6261.71); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6D539AD22+56930]
	(No symbol) [0x00007FF6D530F622]
	(No symbol) [0x00007FF6D51C42E5]
	(No symbol) [0x00007FF6D52098ED]
	(No symbol) [0x00007FF6D5209A2C]
	(No symbol) [0x00007FF6D524A967]
	(No symbol) [0x00007FF6D522BCDF]
	(No symbol) [0x00007FF6D52481E2]
	(No symbol) [0x00007FF6D522BA43]
	(No symbol) [0x00007FF6D51FD438]
	(No symbol) [0x00007FF6D51FE4D1]
	GetHandleVerifier [0x00007FF6D5716AAD+3709933]
	GetHandleVerifier [0x00007FF6D576FFED+4075821]
	GetHandleVerifier [0x00007FF6D576817F+4043455]
	GetHandleVerifier [0x00007FF6D5439756+706710]
	(No symbol) [0x00007FF6D531B8FF]
	(No symbol) [0x00007FF6D5316AE4]
	(N

In [15]:
len(table_list)

67